## eye-tracking experiments

In [103]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder

variables

In [105]:
features = [
    'Category Right',
    'Category Left',
    'Point of Regard Right X [px]',
    'Point of Regard Right Y [px]',
    'Point of Regard Left X [px]',
    'Point of Regard Left Y [px]'
]
categorical = {
    'Category Right': ['Blink', 'Saccade', 'Fixation'],
    'Category Left': ['Blink', 'Saccade', 'Fixation']
}

load data

In [174]:
dfs = []
for file in os.listdir('split/'):
    dfs.append(pd.read_csv('split/' + file))
dfs[24]

C:\Users\User\AppData\Local\Temp\ipykernel_14708\275031605.py:3: DtypeWarning: Columns (14,15,16,17,18,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv('split/' + file))
C:\Users\User\AppData\Local\Temp\ipykernel_14708\275031605.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv('split/' + file))
C:\Users\User\AppData\Local\Temp\ipykernel_14708\275031605.py:3: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv('split/' + file))


,Unnamed: 0.1,Unnamed: 0,RecordingTime [ms],Time of Day [h:m:s:ms],Trial,Stimulus,Export Start Trial Time [ms],Export End Trial Time [ms],Participant,Color,...,Pupil Position Left X [px],Pupil Position Left Y [px],Annotation Name,Annotation Description,Annotation Tags,Mouse Position X [px],Mouse Position Y [px],Scroll Direction X,Scroll Direction Y,Content
0,65851,65851,190230.574,14:14:05:674,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,-,-,-,-,-,-,-,-,-,8 voc chien d.jpg
1,65852,65852,190247.511,14:14:05:691,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,378.3705,332.5000,-,-,-,-,-,-,-,-
2,65853,65853,190267.382,14:14:05:711,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,378.3042,332.6535,-,-,-,-,-,-,-,-
3,65854,65854,190287.381,14:14:05:731,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,378.1739,332.8933,-,-,-,-,-,-,-,-
4,65855,65855,190307.253,14:14:05:751,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,378.0615,332.9575,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,65992,65992,193033.693,14:14:08:477,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,373.7790,331.8850,-,-,-,-,-,-,-,-
142,65993,65993,193053.578,14:14:08:497,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,373.8568,331.7051,-,-,-,-,-,-,-,-
143,65994,65994,193073.457,14:14:08:517,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,374.1514,331.4526,-,-,-,-,-,-,-,-
144,65995,65995,193093.326,14:14:08:537,Trial009,8 voc chien D.jpg,0.0,2882.751,37,LawnGreen,...,374.4949,331.4545,-,-,-,-,-,-,-,-


In [45]:
# optional
instances = pd.concat([df[['Participant', 'Stimulus']].drop_duplicates() for df in dfs], ignore_index=True)
instances

,Participant,Stimulus
0,18,20 eye tracking (ballon droite).avi
1,28,20 eye tracking (ballon droite).avi
2,42,20 eye tracking (ballon droite).avi
3,41,01 neutre2.avi
4,30,2 devant.jpg
...,...,...
2904,30,14 devant point chat gauche.jpg
2905,30,15 devant - Copie.jpg
2906,30,16 voc chat gauche.jpg
2907,30,17 voc devant.jpg


In [11]:
# optional
chopped = []
for df in dfs:
    grouped = df.groupby(['Participant', 'Stimulus'])
    for name, group in grouped:
        chopped.append(group[['Participant', 'Stimulus']])
        # print(group[['Participant', 'Stimulus']])

[748 rows x 2 columns]
      Participant              Stimulus
2643           20  26 b joie triste.jpg
2644           20  26 b joie triste.jpg
2645           20  26 b joie triste.jpg
2646           20  26 b joie triste.jpg
2647           20  26 b joie triste.jpg
...           ...                   ...
2898           20  26 b joie triste.jpg
2899           20  26 b joie triste.jpg
2900           20  26 b joie triste.jpg
2901           20  26 b joie triste.jpg
2902           20  26 b joie triste.jpg

[260 rows x 2 columns]
      Participant                Stimulus
2903           20  27 cadeau dernier2.avi
2904           20  27 cadeau dernier2.avi
2905           20  27 cadeau dernier2.avi
2906           20  27 cadeau dernier2.avi
2907           20  27 cadeau dernier2.avi
...           ...                     ...
3744           20  27 cadeau dernier2.avi
3745           20  27 cadeau dernier2.avi
3746           20  27 cadeau dernier2.avi
3747           20  27 cadeau dernier2.avi
3748       

[170 rows x 2 columns]
      Participant                     Stimulus
86926          15  23 vole triste vs joie4.avi
86927          15  23 vole triste vs joie4.avi
86928          15  23 vole triste vs joie4.avi
86929          15  23 vole triste vs joie4.avi
86930          15  23 vole triste vs joie4.avi
...           ...                          ...
92098          15  23 vole triste vs joie4.avi
92099          15  23 vole triste vs joie4.avi
92100          15  23 vole triste vs joie4.avi
92101          15  23 vole triste vs joie4.avi
92102          15  23 vole triste vs joie4.avi

[1272 rows x 2 columns]
      Participant              Stimulus
87392          15  24 b joie triste.jpg
87393          15  24 b joie triste.jpg
87394          15  24 b joie triste.jpg
87395          15  24 b joie triste.jpg
87396          15  24 b joie triste.jpg
...           ...                   ...
92235          15  24 b joie triste.jpg
92236          15  24 b joie triste.jpg
92237          15  24 b joie

      Participant              Stimulus
80621          27  26 a triste joie.jpg
80622          27  26 a triste joie.jpg
80623          27  26 a triste joie.jpg
80624          27  26 a triste joie.jpg
80625          27  26 a triste joie.jpg
...           ...                   ...
80758          27  26 a triste joie.jpg
80759          27  26 a triste joie.jpg
80760          27  26 a triste joie.jpg
80761          27  26 a triste joie.jpg
80762          27  26 a triste joie.jpg

[142 rows x 2 columns]
      Participant                      Stimulus
80763          27  27 tombe joie vs triste5.avi
80764          27  27 tombe joie vs triste5.avi
80765          27  27 tombe joie vs triste5.avi
80766          27  27 tombe joie vs triste5.avi
80767          27  27 tombe joie vs triste5.avi
...           ...                           ...
81389          27  27 tombe joie vs triste5.avi
81390          27  27 tombe joie vs triste5.avi
81391          27  27 tombe joie vs triste5.avi
81392          2

     Participant         Stimulus
2945          22  01 neutre22.avi
2946          22  01 neutre22.avi
2947          22  01 neutre22.avi
2948          22  01 neutre22.avi
2949          22  01 neutre22.avi
...          ...              ...
3925          22  01 neutre22.avi
3926          22  01 neutre22.avi
3927          22  01 neutre22.avi
3928          22  01 neutre22.avi
3929          22  01 neutre22.avi

[985 rows x 2 columns]
       Participant           Stimulus
340093          22  A Triste joie.jpg
340094          22  A Triste joie.jpg
340095          22  A Triste joie.jpg
340096          22  A Triste joie.jpg
340097          22  A Triste joie.jpg
...            ...                ...
340448          22  A Triste joie.jpg
340449          22  A Triste joie.jpg
340450          22  A Triste joie.jpg
340451          22  A Triste joie.jpg
340452          22  A Triste joie.jpg

[360 rows x 2 columns]
       Participant           Stimulus
171266          22  A joie Triste.jpg
171267      

       Participant           Stimulus
101297          54  B joie Triste.jpg
101298          54  B joie Triste.jpg
101299          54  B joie Triste.jpg
101300          54  B joie Triste.jpg
101301          54  B joie Triste.jpg
...            ...                ...
106992          54  B joie Triste.jpg
106993          54  B joie Triste.jpg
106994          54  B joie Triste.jpg
106995          54  B joie Triste.jpg
106996          54  B joie Triste.jpg

[2128 rows x 2 columns]
       Participant                     Stimulus
354928          54  bonbons triste vs joie1.avi
354929          54  bonbons triste vs joie1.avi
354930          54  bonbons triste vs joie1.avi
354931          54  bonbons triste vs joie1.avi
354932          54  bonbons triste vs joie1.avi
...            ...                          ...
362998          54  bonbons triste vs joie1.avi
362999          54  bonbons triste vs joie1.avi
363000          54  bonbons triste vs joie1.avi
363001          54  bonbons triste vs j

...

[146 rows x 2 columns]
      Participant            Stimulus
18229          44  09 voc chien d.jpg
18230          44  09 voc chien d.jpg
18231          44  09 voc chien d.jpg
18232          44  09 voc chien d.jpg
18233          44  09 voc chien d.jpg
...           ...                 ...
18371          44  09 voc chien d.jpg
18372          44  09 voc chien d.jpg
18373          44  09 voc chien d.jpg
18374          44  09 voc chien d.jpg
18375          44  09 voc chien d.jpg

[147 rows x 2 columns]
      Participant           Stimulus
18376          44  10 voc devant.jpg
18377          44  10 voc devant.jpg
18378          44  10 voc devant.jpg
18379          44  10 voc devant.jpg
18380          44  10 voc devant.jpg
...           ...                ...
18518          44  10 voc devant.jpg
18519          44  10 voc devant.jpg
18520          44  10 voc devant.jpg
18521          44  10 voc devant.jpg
18522          44  10 voc devant.jpg

[147 rows x 2 columns]
      Participant       Stimu

[134 rows x 2 columns]
      Participant           Stimulus
12625          49  18 voc devant.jpg
12626          49  18 voc devant.jpg
12627          49  18 voc devant.jpg
12628          49  18 voc devant.jpg
12629          49  18 voc devant.jpg
...           ...                ...
12767          49  18 voc devant.jpg
12768          49  18 voc devant.jpg
12769          49  18 voc devant.jpg
12770          49  18 voc devant.jpg
12771          49  18 voc devant.jpg

[147 rows x 2 columns]
      Participant         Stimulus
12772          49  19 aurevoir.jpg
12773          49  19 aurevoir.jpg
12774          49  19 aurevoir.jpg
12775          49  19 aurevoir.jpg
12776          49  19 aurevoir.jpg
...           ...              ...
12913          49  19 aurevoir.jpg
12914          49  19 aurevoir.jpg
12915          49  19 aurevoir.jpg
12916          49  19 aurevoir.jpg
12917          49  19 aurevoir.jpg

[146 rows x 2 columns]
    Participant         Stimulus
0            50  02 coucou d.jpg

[118 rows x 2 columns]
      Participant                          Stimulus
37319          13  14 tete pointage chat gauche.jpg
37320          13  14 tete pointage chat gauche.jpg
37321          13  14 tete pointage chat gauche.jpg
37322          13  14 tete pointage chat gauche.jpg
37323          13  14 tete pointage chat gauche.jpg
...           ...                               ...
37436          13  14 tete pointage chat gauche.jpg
37437          13  14 tete pointage chat gauche.jpg
37438          13  14 tete pointage chat gauche.jpg
37439          13  14 tete pointage chat gauche.jpg
37440          13  14 tete pointage chat gauche.jpg

[122 rows x 2 columns]
      Participant                         Stimulus
37441          13  15 devant point chat gauche.jpg
37442          13  15 devant point chat gauche.jpg
37443          13  15 devant point chat gauche.jpg
37444          13  15 devant point chat gauche.jpg
37445          13  15 devant point chat gauche.jpg
...           ...      

[164 rows x 2 columns]
             Participant                 Stimulus
33809  Unidentified(Neg)  12 yeux chat gauche.jpg
33810  Unidentified(Neg)  12 yeux chat gauche.jpg
33811  Unidentified(Neg)  12 yeux chat gauche.jpg
33812  Unidentified(Neg)  12 yeux chat gauche.jpg
33813  Unidentified(Neg)  12 yeux chat gauche.jpg
...                  ...                      ...
35201  Unidentified(Neg)  12 yeux chat gauche.jpg
35202  Unidentified(Neg)  12 yeux chat gauche.jpg
35203  Unidentified(Neg)  12 yeux chat gauche.jpg
35204  Unidentified(Neg)  12 yeux chat gauche.jpg
35205  Unidentified(Neg)  12 yeux chat gauche.jpg

[121 rows x 2 columns]
             Participant                 Stimulus
33873  Unidentified(Neg)  13 tete chat gauche.jpg
33874  Unidentified(Neg)  13 tete chat gauche.jpg
33875  Unidentified(Neg)  13 tete chat gauche.jpg
33876  Unidentified(Neg)  13 tete chat gauche.jpg
33877  Unidentified(Neg)  13 tete chat gauche.jpg
...                  ...                      ...
352

      Participant       Stimulus
71810          23  FEDE Drte.avi
71811          23  FEDE Drte.avi
71812          23  FEDE Drte.avi
71813          23  FEDE Drte.avi
71814          23  FEDE Drte.avi
...           ...            ...
73138          23  FEDE Drte.avi
73139          23  FEDE Drte.avi
73140          23  FEDE Drte.avi
73141          23  FEDE Drte.avi
73142          23  FEDE Drte.avi

[1333 rows x 2 columns]
      Participant       Stimulus
73143          27  FEDE Drte.avi
73144          27  FEDE Drte.avi
73145          27  FEDE Drte.avi
73146          27  FEDE Drte.avi
73147          27  FEDE Drte.avi
...           ...            ...
74474          27  FEDE Drte.avi
74475          27  FEDE Drte.avi
74476          27  FEDE Drte.avi
74477          27  FEDE Drte.avi
74478          27  FEDE Drte.avi

[1336 rows x 2 columns]
      Participant       Stimulus
70058          30  FEDE Drte.avi
70059          30  FEDE Drte.avi
70060          30  FEDE Drte.avi
70061          30  FEDE Dr

In [12]:
# optional
len(chopped)

2909

Participant data

In [108]:
participants = pd.read_csv('Metadata_Participants.csv')

classes

In [175]:
class experiment:
    def __init__(self, df, parti, stimu, asd):
        self.data = df
        self.participant = parti
        self.stimulus = stimu
        self.asd = (asd == 'ASD')
        
        # extract other frame-independent info
        self.trackRatio = self.data['Tracking Ratio [%]'][0]

    def preprocess(self):

        # delete columns that are information outputs
        self.data = self.data.drop(index=0)
        self.data = self.data.reset_index(drop=True)
        
        # drop the unused columns
        fs = list(self.data.columns)
        for f in fs:
            if f not in features: self.data.drop(columns=[f], inplace=True)

        # some data are missing Category Left. Copy Category Right over if that's the case.
        if 'Category Left' not in self.data.columns:
            self.data['Category Left'] = self.data['Category Right']
    
        # one hot encoding
        for category in categorical:
            self.data[[category]] = self.data[[category]].replace('-', 'Blink') # replace all unwanted categories with `Blink`
            self.data[[category]] = self.data[[category]].replace('Separator', 'Blink')
            self.data[[category]] = self.data[[category]].replace('Left Click', 'Blink')
            self.data[[category]] = self.data[[category]].replace('Right Click', 'Blink')
            encoder = OneHotEncoder(categories=[categorical[category]])
            OHE = encoder.fit_transform(self.data[[category]]).toarray()
            OHE = pd.DataFrame(OHE, columns=encoder.get_feature_names_out())
            self.data = pd.concat([self.data, OHE])

        # drop columns that are not needed anymore
        # self.data = self.data.loc[:, ~self.data.columns.duplicated()]
        self.data.drop(columns=list(categorical.keys()), inplace=True)

In [176]:
experiments = []

for df in dfs:

    # participant
    p = df['Participant'][0]
    if p == 'Unidentified(Neg)': p = -1
    elif p == 'Unidentified(Pos)': p = 0

    # stimulus
    s = df['Stimulus'][0]

    # class
    if p == -1: c = 'TD'
    elif p == 0: c = 'ASD'
    else: c = participants['Class'][p - 1]

    # call constructor
    experiments.append(experiment(df, p, s, c))

In [177]:
c = 0
for exp in experiments:
    print(c)
    exp.preprocess()
    c += 1

0


ValueError: Found unknown categories [nan] in column 0 during fit

In [172]:
experiments[2339].data

,Point of Regard Right X [px],Point of Regard Right Y [px],Point of Regard Left X [px],Point of Regard Left Y [px]
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
2716,0,0,0,0
2717,0,0,0,0
2718,0,0,0,0
2719,0,0,0,0


In [171]:
for exp in experiments: print(len(exp.data.columns))

4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


""


In [7]:
master = pd.concat([df[['Participant', 'Stimulus']].drop_duplicates() for df in dfs], ignore_index=True)

In [8]:
master

,Participant,Stimulus
0,50,20 eye tracking (ballon droite).avi
1,52,20 eye tracking (ballon droite).avi
2,49,20 eye tracking (ballon droite).avi
3,55,20 eye tracking (ballon droite).avi
4,53,20 eye tracking (ballon droite).avi
...,...,...
2904,39,FEDE Drte.avi
2905,37,FEDE Drte.avi
2906,30,FEDE Drte.avi
2907,23,FEDE Drte.avi


In [9]:
master = master.applymap(str).drop_duplicates()

In [10]:
master

,Participant,Stimulus
0,50,20 eye tracking (ballon droite).avi
1,52,20 eye tracking (ballon droite).avi
2,49,20 eye tracking (ballon droite).avi
3,55,20 eye tracking (ballon droite).avi
4,53,20 eye tracking (ballon droite).avi
...,...,...
2903,39,FEDE Drte.avi
2905,37,FEDE Drte.avi
2906,30,FEDE Drte.avi
2907,23,FEDE Drte.avi


In [26]:
master = master.applymap(str).drop_duplicates()

master

,Participant,Trial
0,50,Trial020
1,52,Trial020
2,49,Trial020
3,55,Trial020
4,53,Trial020
...,...,...
2912,24,Trial001
2919,25,Trial019
2920,26,Trial019
3231,Unidentified(Pos),Trial010


In [27]:
master['Participant'].unique()

array(['50', '52', '49', '55', '53', '46', '42', 'Unidentified(Neg)',
       'Unidentified(Pos)', '20', '19', '27', '17', '15', '21', '23',
       '28', '18', '43', '48', '51', '44', '45', '47', '13', '14', '54',
       '59', '25', '26', '58', '29', '56', '30', '57', '31', '36', '2',
       '3', '4', '5', '6', '1', '9', '10', '8', '34', '40', '38', '32',
       '35', '11', '33', '39', '37', '7', '41', '22', '24'], dtype=object)